In [ ]:
import pandas

Answer = pandas.read_sql('Answer_trimmed', "sqlite:///data/QAT_trimmed.db", index_col='index')
Question = pandas.read_sql('Question_trimmed', "sqlite:///data/QAT_trimmed.db", index_col='index')


In [ ]:
Answer_sorted_score = Answer.sort_values(by=['ParentId','Score','CreationDate_to_datetime'],ascending=[1,0,1])
Answer_sorted_time = Answer.sort_values(by=['ParentId','CreationDate_to_datetime'])
Question_sorted_Id = Question.sort_values(by=['Id'])

In [ ]:
time_gen = Answer_sorted_time.iterrows()
score_gen = Answer_sorted_score.iterrows()
count = 0
next_score_data = next(score_gen)
next_time_data = next(time_gen)
for idx, row in Question_sorted_Id.iterrows():
    Question_sorted_Id.at[idx,'BestAnsTime'] = next_score_data[1]['CreationDate_to_datetime']
    Question_sorted_Id.at[idx,'FirstAnsTime'] = next_time_data[1]['CreationDate_to_datetime']
    next_score_data = next(score_gen)
    next_time_data = next(time_gen)
    while(next_score_data[1]['ParentId'] == row['Id']):
        try:
            next_score_data = next(score_gen)
            next_time_data = next(time_gen)
        except StopIteration:
            print('\nfinished')
            break
    if count % 10000 == 0:
        print('#', end='')
    count+=1


In [ ]:
Question_sorted_Id = Question_sorted_Id.drop(columns=['OwnerUserId','ClosedDate'])
Question_sorted_Id.to_sql('Question', "sqlite:///data/QAT_final.db",  if_exists = "replace")